In [18]:
!pip uninstall torch torchvision torchaudio

^C


In [19]:
!pip install transformers datasets scikit-learn

In [20]:
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cpu

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cpu


In [21]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from datasets import Dataset
import json
# Load the dataset
file_path = 'Training.csv'
data = pd.read_csv(file_path)

# Parse the JSON-like strings in the justification fields
def parse_json_field(field):
    try:
        return json.loads(field.replace("'", "\""))
    except json.JSONDecodeError:
        return {}

# Apply the parsing function to the relevant columns
data['justification_openai_parsed'] = data['justification_openai'].apply(parse_json_field)

# Extract the text field for fine-tuning
data['justification_openai_text'] = data['justification_openai_parsed'].apply(lambda x: x.get('justification', ''))

# Use the score as the label
data['label'] = data['score_openai'] - 1  # Assuming scores are from 1 to 5, adjust to 0-indexed

# Train-test split
train_df, test_df = train_test_split(data[['justification_openai_text', 'label']], test_size=0.2, random_state=42)

# Convert to Hugging Face Dataset objects
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)


OSError: [WinError 126] The specified module could not be found. Error loading "d:\Mercor\.venv\Lib\site-packages\torch\lib\fbgemm.dll" or one of its dependencies.

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize_function(examples):
    return tokenizer(examples['justification_openai_text'], padding="max_length", truncation=True)

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Set the format for PyTorch tensors
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/4819 [00:00<?, ? examples/s]

Map:   0%|          | 0/1205 [00:00<?, ? examples/s]

In [5]:
# Load the pre-trained BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=5)  # 5 classes

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Train the model
trainer.train()


In [ ]:
# Evaluate the model on the test set
eval_result = trainer.evaluate()
print(f"Evaluation Result: {eval_result}")


In [ ]:
# Save the fine-tuned model
model.save_pretrained('./fine_tuned_bert_model')
tokenizer.save_pretrained('./fine_tuned_bert_model')


In [ ]:
# Example inference on new data
text = "The candidate uses technical terminology accurately."
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
outputs = model(**inputs)
predicted_class = torch.argmax(outputs.logits, dim=1).item() + 1  # Add 1 to convert back to 1-indexed
print(f"Predicted Score: {predicted_class}")
